In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
import random
import numpy as np

# Разбиение обучающих данных
# labels_num -  количество меток (от 0 до 9)
# train_len - длина того обучающего множества, которое мы хотим использовать для обучения, оно должно делиться на количество меток
def balancing(full_train_dataset, lables_num, train_len):
  # Длина всего обучающего множества и того обучающего множества, которое мы хотим использовать для обучения
  full_train_len = len(full_train_dataset)
  # Количество данных с одной меткой
  label_group_num = int(train_len/lables_num)

  # Создаём группы для хранения индексов каждой метки в обучающем наборе данных
  label_groups_index = [[] for _ in range(lables_num)]
  for i in range(full_train_len):
    label = full_train_dataset[i][1]
    label_groups_index[label].append(i)

  # Обрезаем группы, оставляя случайные, неповторяющиеся элементы в каждой и объединяем их всех в один набор индексов
  all_index = np.array([], dtype=int)
  for i in range(lables_num):
    all_index = np.append(all_index, random.sample(label_groups_index[i], label_group_num))
  np.random.shuffle(all_index)

  # Формируем обучающий набор данных
  train_dataset = torch.utils.data.Subset(full_train_dataset, all_index)
  return train_dataset

In [19]:
# Определение модели
# neurons_num - количество нейронов в каждом слое
# img_size - размер изображений из MNIST
class Net(nn.Module):
    def __init__(self, neurons_num, img_size):
        super(Net, self).__init__()
        self.img_size = img_size
        self.fc1 = nn.Linear(img_size, neurons_num)
        self.fc2 = nn.Linear(neurons_num, neurons_num)
        self.fc3 = nn.Linear(neurons_num, neurons_num)
        self.fc4 = nn.Linear(neurons_num, neurons_num)
        self.fc5 = nn.Linear(neurons_num, neurons_num)
        self.fc6 = nn.Linear(neurons_num, 10)

    def forward(self, x):
        x = x.view(-1, self.img_size)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # вывод текущего прогресса, для того, чтобы убедиться, что обучение идёт
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
# Загрузка данных MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 13201693.74it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28869326.46it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5847099.99it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [7]:
train_dataset = balancing(train_dataset, 10, 30000)

In [23]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [31]:
# Инициализация модели 
model = Net(1000, 28*28).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [32]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302168  [   64/30000]
loss: 2.292595  [ 6464/30000]
loss: 2.255351  [12864/30000]
loss: 1.079496  [19264/30000]
loss: 1.094166  [25664/30000]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.511084 

Test Error: 
 Accuracy: 85.0%, Avg loss: 0.491694 

Epoch 2
-------------------------------
loss: 0.719588  [   64/30000]
loss: 0.566483  [ 6464/30000]
loss: 0.459434  [12864/30000]
loss: 0.216495  [19264/30000]
loss: 0.355677  [25664/30000]
Test Error: 
 Accuracy: 94.9%, Avg loss: 0.167971 

Test Error: 
 Accuracy: 94.5%, Avg loss: 0.180327 

Epoch 3
-------------------------------
loss: 0.289856  [   64/30000]
loss: 0.301145  [ 6464/30000]
loss: 0.276715  [12864/30000]
loss: 0.051749  [19264/30000]
loss: 0.143000  [25664/30000]
Test Error: 
 Accuracy: 97.0%, Avg loss: 0.100583 

Test Error: 
 Accuracy: 95.9%, Avg loss: 0.135033 

Epoch 4
-------------------------------
loss: 0.152150  [   64/30000]
loss: 0.226655  [ 6464/30000]
loss: 0.189525  [1